In [4]:
import pandas as pd
import statsmodels.api as sm

# 1. 读取处理后的数据
df = pd.read_csv('data_processed.csv')

# 2. 设置因变量（目标变量）
target_col = 'Y染色体浓度'  # 你可以改成其他目标变量

# 3. 选择数值型自变量（排除目标列）
numeric_cols = ['年龄', '孕妇BMI', '检测孕周_周数']

# 4. 定义 X 和 y
X = df[numeric_cols]
y = df[target_col]

# 5. 添加常数项（截距）
X = sm.add_constant(X)

# 6. 建立 OLS 回归模型
model = sm.OLS(y, X).fit()

# 7. 输出回归结果
print(model.summary())

# 8. 提取显著性变量（p < 0.05）
p_values = model.pvalues
significant_vars = p_values[p_values < 0.05].index.tolist()

# 去掉常数项
if 'const' in significant_vars:
    significant_vars.remove('const')

print("\n显著性变量（p < 0.05）：")
print(significant_vars)

# 9. 保存回归结果到文件
summary_df = pd.DataFrame({
    '变量': model.params.index,
    '系数': model.params.values,
    'p值': model.pvalues.values,
    '显著性': ['显著' if p < 0.05 else '不显著' for p in model.pvalues.values]
})
summary_df.to_csv('regression_results.csv', index=False, encoding='utf-8-sig')

print("\n回归结果已保存到 regression_results.csv")


                            OLS Regression Results                            
Dep. Variable:                 Y染色体浓度   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     15.30
Date:                Fri, 05 Sep 2025   Prob (F-statistic):           9.99e-10
Time:                        11:32:53   Log-Likelihood:                -1234.2
No. Observations:                 938   AIC:                             2476.
Df Residuals:                     934   BIC:                             2496.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0285      0.030      0.965      0.3

## 第二次分析:

这种现象在实际数据分析中很常见，原因可能有：

多重共线性（Multicollinearity）

BMI 和其他变量（如身高、体重）高度相关
在 OLS 中，如果自变量之间高度相关，系数估计会不稳定，p 值会变大
可能 BMI 的信息已经被“身高+体重”解释掉了
模型设定不当

OLS 假设是线性关系，但 BMI 和因变量可能是非线性关系
如果真实关系是曲线型，线性模型会低估它的显著性
噪声和样本量

样本量不足或噪声较大，也会导致 p 值偏大
多重比较问题

你有很多变量（>20 个），在多次显著性检验中，必然会有一些变量 p 值不显著，即使它们在实际中有影响


### 方差膨胀因子（VIF）:

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

import pandas as pd
import statsmodels.api as sm

# 1. 读取处理后的数据
df = pd.read_csv('data_processed.csv')

X = df.drop(columns=['Y染色体浓度'])  # 自变量
X = sm.add_constant(X)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

         feature            VIF
0          const  252425.921793
1             年龄       1.125899
2             身高     148.599371
3             体重     408.539086
4           末次月经      73.733558
5          IVF妊娠       1.060238
6           检测日期      85.476486
7         检测抽血次数       2.568300
8          孕妇BMI     236.741013
9          原始读段数       1.461448
10  在参考基因组上比对的比例       1.064944
11       重复读段的比例       1.102555
12      唯一比对的读段数       1.468183
13          GC含量       1.696878
14     13号染色体的Z值       1.303527
15     18号染色体的Z值       1.406597
16     21号染色体的Z值       1.070954
17       X染色体的Z值       2.051902
18       Y染色体的Z值       1.757459
19        X染色体浓度       1.290843
20   13号染色体的GC含量       1.772619
21   18号染色体的GC含量       1.667991
22   21号染色体的GC含量       1.619536
23    被过滤掉读段数的比例       1.141593
24          怀孕次数       1.789669
25          生产次数       1.742485
26        胎儿是否健康       1.127160
27       检测孕周_周数       5.116699


### LassoCV

In [ ]:
from sklearn.linear_model import LassoCV
model = LassoCV(cv=5).fit(X, y)
coef = pd.Series(model.coef_, index=X.columns)
print(coef)


const           0.000000e+00
年龄             -0.000000e+00
身高             -0.000000e+00
体重             -0.000000e+00
末次月经           -8.627704e-10
IVF妊娠           0.000000e+00
检测日期            0.000000e+00
检测抽血次数          0.000000e+00
孕妇BMI          -0.000000e+00
原始读段数          -0.000000e+00
在参考基因组上比对的比例   -0.000000e+00
重复读段的比例         0.000000e+00
唯一比对的读段数       -0.000000e+00
GC含量           -0.000000e+00
13号染色体的Z值      -0.000000e+00
18号染色体的Z值      -0.000000e+00
21号染色体的Z值       0.000000e+00
X染色体的Z值        -0.000000e+00
Y染色体的Z值         0.000000e+00
X染色体浓度          0.000000e+00
13号染色体的GC含量    -0.000000e+00
18号染色体的GC含量    -0.000000e+00
21号染色体的GC含量    -0.000000e+00
被过滤掉读段数的比例      0.000000e+00
怀孕次数           -0.000000e+00
生产次数            0.000000e+00
胎儿是否健康          0.000000e+00
检测孕周_周数         0.000000e+00
dtype: float64


### 用 随机森林 计算特征重要性

In [25]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
X = X.drop(['const'], axis=1)
rf.fit(X, y)
importance = pd.Series(rf.feature_importances_, index=X.columns)
print(importance.sort_values(ascending=False))

X染色体浓度          0.322120
检测日期            0.129149
Y染色体的Z值         0.075713
检测抽血次数          0.070651
检测孕周_周数         0.068856
孕妇BMI           0.050454
X染色体的Z值         0.037723
末次月经            0.024920
被过滤掉读段数的比例      0.024850
18号染色体的Z值       0.020842
在参考基因组上比对的比例    0.019056
GC含量            0.016015
13号染色体的GC含量     0.015601
21号染色体的GC含量     0.015131
唯一比对的读段数        0.015127
重复读段的比例         0.014959
21号染色体的Z值       0.014843
13号染色体的Z值       0.014217
18号染色体的GC含量     0.014034
年龄              0.013128
原始读段数           0.012927
生产次数            0.003989
怀孕次数            0.003784
胎儿是否健康          0.001611
IVF妊娠           0.000300
dtype: float64


这边可以看到，以0.01作为判别，我们可以取出下面的变量

In [ ]:
# 取出importance.sort_values(ascending=False)中大于0.01的
filtered_importance = importance.sort_values(ascending=False)[importance > 0.01]
filtered_importance.keys()

Index(['X染色体浓度', '检测日期', 'Y染色体的Z值', '检测孕周_周数', '检测抽血次数', '孕妇BMI', 'X染色体的Z值',
       '末次月经', '被过滤掉读段数的比例', '18号染色体的Z值', '在参考基因组上比对的比例', '13号染色体的Z值',
       '21号染色体的GC含量', '13号染色体的GC含量', '21号染色体的Z值', 'GC含量', '重复读段的比例', '原始读段数',
       '唯一比对的读段数', '18号染色体的GC含量', '年龄'],
      dtype='object')